In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE25307"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE25307"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE25307.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE25307.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE25307.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE25307_family.soft.gz', 'GSE25307_series_matrix.txt.gz']
Identified SOFT files: ['GSE25307_family.soft.gz']
Identified matrix files: ['GSE25307_series_matrix.txt.gz']



Background Information:
!Series_title	"The retinoblastoma gene is targeted for rearrangements in BRCA1-deficient basal-like breast cancer."
!Series_summary	"Breast tumors from BRCA1 germ line mutation carriers typically exhibit features of the basal-like molecular subtype. However, the specific genes recurrently mutated as a consequence of BRCA1 dysfunction have not been fully elucidated. In this study, we utilized gene expression profiling to molecularly subtype 577 breast tumors, including 73 breast tumors from BRCA1/2 mutation carriers. Focusing on the RB1 locus, we analyzed 33 BRCA1-mutated, 36 BRCA2-mutated and 48 non-BRCA1/2-mutated breast tumors using a custom-designed high-density oligomicroarray covering the RB1 gene. We found a strong association between the basal-like subtype and BRCA1-mutated breast tumors and the luminal B subtype and BRCA2-mutated breast tumors. RB1 was identified as a major target for genomic disruption in tumors arising in BRCA1 mutation carriers and i

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Yes, this dataset contains gene expression data (based on the Series_summary/design)
is_gene_available = True

# 2.1 Data Availability
# For trait - we identify retinoblastoma from BRCA status
trait_row = 10  # "brca status" values in index 10
# Age is not available in sample characteristics
age_row = None
# Gender is not available in sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary based on BRCA1 status"""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
        if value.lower() == 'brca1':
            return 1  # BRCA1 mutated is our target condition
        elif value.lower() in ['sporadic', 'brca2', 'brcax', 'other']:
            return 0  # Non-BRCA1 as control
    return None

def convert_age(value):
    """Convert age to continuous value"""
    # Not used since age data is not available, but function is defined for interface
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
        try:
            return float(value)
        except ValueError:
            pass
    return None

def convert_gender(value):
    """Convert gender to binary value"""
    # Not used since gender data is not available, but function is defined for interface
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if value in ['female', 'f']:
            return 0
        elif value in ['male', 'm']:
            return 1
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is available from a previous step
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


Preview of clinical data:
{'GSM551608': [0.0], 'GSM551609': [0.0], 'GSM551610': [0.0], 'GSM551611': [0.0], 'GSM551612': [0.0], 'GSM551613': [0.0], 'GSM551614': [0.0], 'GSM551615': [0.0], 'GSM551616': [0.0], 'GSM551617': [1.0], 'GSM551618': [0.0], 'GSM551619': [0.0], 'GSM551620': [0.0], 'GSM551621': [0.0], 'GSM551622': [0.0], 'GSM551623': [0.0], 'GSM551624': [0.0], 'GSM551625': [0.0], 'GSM551626': [0.0], 'GSM551627': [0.0], 'GSM551628': [0.0], 'GSM551629': [0.0], 'GSM551630': [0.0], 'GSM551631': [1.0], 'GSM551632': [0.0], 'GSM551633': [0.0], 'GSM551634': [0.0], 'GSM551635': [0.0], 'GSM551636': [0.0], 'GSM551637': [0.0], 'GSM551638': [0.0], 'GSM551639': [0.0], 'GSM551640': [0.0], 'GSM551641': [0.0], 'GSM551642': [0.0], 'GSM551643': [0.0], 'GSM551644': [0.0], 'GSM551645': [0.0], 'GSM551646': [0.0], 'GSM551647': [0.0], 'GSM551648': [0.0], 'GSM551649': [1.0], 'GSM551650': [0.0], 'GSM551651': [0.0], 'GSM551652': [0.0], 'GSM551653': [0.0], 'GSM551654': [0.0], 'GSM551655': [0.0], 'GSM551656': 

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['31', '32', '33', '34', '53', '55', '56', '61', '64', '66', '75', '77',
       '79', '80', '89', '91', '93', '95', '96', '97'],
      dtype='object', name='ID')

Gene expression data shape: (10377, 577)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numeric values rather than standard gene symbols
# Human gene symbols typically follow a standardized nomenclature (e.g., TP53, BRCA1, etc.)
# The numeric identifiers shown (31, 32, 33, etc.) are likely probe IDs or other platform-specific identifiers
# that need to be mapped to proper gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Unicode decoding error: 'utf-8' codec can't decode byte 0xc9 in position 6449: invalid continuation byte
Trying alternative approach...


Gene annotation preview (alternative method):
{'ID': ['1', '2', '3', '4', '5'], 'CLONEID': ['H200021335', 'H200008181', 'H300008355', 'H300007697', 'H300002929'], 'SPOT_ID': ['Operon oligo: H200021335', nan, nan, 'Operon oligo: H300007697', 'TMEM31'], 'SPOT_ID_DESCRIPTION': [nan, nan, nan, nan, nan], 'ReporterID': ['H200021335', 'H200008181', 'H300008355', 'H300007697', 'H300002929'], 'OligoSet_genelist': ['V2.1.3', 'V2.1.3', 'V4.0.2', 'V3.0.3', 'V4.0.2'], 'OligoSet_geneSymbol': [nan, 'EIF4A2', 'NP_078953.2', nan, 'TMEM31'], 'OligoSet_description': [nan, 'Eukaryotic initiation factor 4A-II (eIF4A-II) (eIF-4A-II). [Source:Uniprot/SWISSPROT;Acc:Q14240]', nan, nan, nan], 'OligoSet_ensemblGene': [nan, 'ENSG00000156976', 'ENSG00000179299', nan, 'ENSG00000179363'], 'OligoSet_ensemblTranscript': [nan, 'ENST00000356531,ENST00000323963', 'ENST00000333934;ENST00000381782;ENST00000316607;ENST00000381780', nan, 'ENST00000319560;ENST00000372615'], 'ARRAY_BLOCK': [1.0, 1.0, 1.0, 1.0, 1.0], 'BLOCK_CO

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene IDs and gene symbols in annotation data
# From the preview, 'ID' column seems to contain numeric IDs matching the gene data
# 'OligoSet_geneSymbol' appears to contain gene symbols where available

# Print some samples to verify our mapping choices
print("ID column sample:", gene_annotation['ID'].head(10).tolist())
print("Gene symbol column sample:", gene_annotation['OligoSet_geneSymbol'].head(10).tolist())

# 2. Get gene mapping dataframe
mapping_data = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',
    gene_col='OligoSet_geneSymbol'
)

print("\nMapping data shape:", mapping_data.shape)
print("Mapping data preview:")
print(preview_df(mapping_data))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

print("\nGene expression data shape after mapping:", gene_data.shape)
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())


ID column sample: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Gene symbol column sample: [nan, 'EIF4A2', 'NP_078953.2', nan, 'TMEM31', 'FAM79B', nan, nan, nan, nan]



Mapping data shape: (32962, 2)
Mapping data preview:
{'ID': ['2', '3', '5', '6', '12'], 'Gene': ['EIF4A2', 'NP_078953.2', 'TMEM31', 'FAM79B', 'Q8TB43_HUMAN']}



Gene expression data shape after mapping: (6991, 577)
First 10 gene symbols:
['A4GALT', 'AAAS', 'AACS', 'AADACL1', 'AADACL2', 'AADAT', 'AAMP', 'AARS', 'AASDHPPT', 'AATF']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously extracted clinical data and prepare it for linking
try:
    # Load the clinical data we saved in Step 2
    clinical_data_df = pd.read_csv(out_clinical_data_file)
    print(f"Clinical data shape before preparation: {clinical_data_df.shape}")
    
    # Create a properly formatted clinical dataframe with the trait as a row, not a column
    # This matches what geo_link_clinical_genetic_data expects
    sample_ids = clinical_data_df.columns.tolist()
    trait_values = clinical_data_df.values[0].tolist()  # Get values from first row
    
    # Create a new DataFrame with the right format (samples as columns, traits as rows)
    formatted_clinical_df = pd.DataFrame([trait_values], 
                                        index=[trait], 
                                        columns=sample_ids)
    
    print("Clinical data prepared for linking:")
    print(preview_df(formatted_clinical_df))
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(formatted_clinical_df, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Print the columns to verify trait is present
    print(f"Columns in linked data: {linked_data.columns.tolist()[:5]}...")
    
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Check if trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Validate the quality of the dataset and save metadata
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,  # We have trait data (BRCA1 status)
        is_biased=is_trait_biased,
        df=linked_data,
        note="Dataset contains gene expression data from breast cancer samples with BRCA1 mutation status as the marker for Retinoblastoma gene targeting."
    )
    
    # 6. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data quality check failed. The dataset is not suitable for association studies.")
        
except Exception as e:
    print(f"Error in data linking: {str(e)}")
    print(f"Exception type: {type(e)}")
    # Provide more detail about the error
    import traceback
    traceback.print_exc()
    
    # Fallback plan if there's an issue with the clinical data
    # Create a minimal DataFrame with sample IDs from gene data
    minimal_df = pd.DataFrame(index=normalized_gene_data.columns)
    minimal_df[trait] = np.nan  # Add trait column with NaN values
    
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True,
        is_trait_available=False,  
        is_biased=True, 
        df=minimal_df,
        note="Error occurred during data processing: " + str(e)
    )
    print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")

Normalized gene data shape: (6886, 577)
First few normalized gene symbols: ['A4GALT', 'AAAS', 'AACS', 'AADACL2', 'AADAT', 'AAMP', 'AARS1', 'AASDHPPT', 'AATF', 'ABAT']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE25307.csv
Clinical data shape before preparation: (1, 577)
Clinical data prepared for linking:
{'GSM551608': [0.0], 'GSM551609': [0.0], 'GSM551610': [0.0], 'GSM551611': [0.0], 'GSM551612': [0.0], 'GSM551613': [0.0], 'GSM551614': [0.0], 'GSM551615': [0.0], 'GSM551616': [0.0], 'GSM551617': [1.0], 'GSM551618': [0.0], 'GSM551619': [0.0], 'GSM551620': [0.0], 'GSM551621': [0.0], 'GSM551622': [0.0], 'GSM551623': [0.0], 'GSM551624': [0.0], 'GSM551625': [0.0], 'GSM551626': [0.0], 'GSM551627': [0.0], 'GSM551628': [0.0], 'GSM551629': [0.0], 'GSM551630': [0.0], 'GSM551631': [1.0], 'GSM551632': [0.0], 'GSM551633': [0.0], 'GSM551634': [0.0], 'GSM551635': [0.0], 'GSM551636': [0.0], 'GSM551637': [0.0], 'GSM551638': [0.0], 'GSM551639': [0.0], 'GSM551640': [0.0], 'GSM551641': [0.0], 'GSM551642': [0.0], 'GSM551643': [0.0], 'GSM551644': [0.0], 'GSM551645': [0.0], 'GSM551646': [0.0], 'GSM551647': [0.0], 'GSM551648': [0.0], 'G

Linked data shape after handling missing values: (359, 6887)
For the feature 'Retinoblastoma', the least common label is '1.0' with 21 occurrences. This represents 5.85% of the dataset.
The distribution of the feature 'Retinoblastoma' in this dataset is fine.



Linked data saved to ../../output/preprocess/Retinoblastoma/GSE25307.csv
